In [1]:
import librosa
import os,glob,pickle
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import soundfile as sf

In [108]:
import tensorflow as tf


In [2]:
from audiomentations import Compose, AddGaussianNoise, PitchShift, HighPassFilter

In [136]:
def featureExtractor(audio):
    librosa_audio_data, librosa_sample_rate=librosa.load(audio)
    mfccs_features= librosa.feature.mfcc(y=librosa_audio_data, sr= librosa_sample_rate,n_mfcc=50)
    ar=np.resize(feature,(50,280))
    ar = np.expand_dims(ar, axis=-1)
#     mfccs_features=mfccs_features.reshape(50,280)
#     print(librosa_sample_rate)
    return ar

In [4]:
def featureExtractor2(audio,sr):
    mfccs_features= librosa.feature.mfcc(y=audio, sr= sr,n_mfcc=50)
    return mfccs_features

In [6]:
x=[]
y=[]
def load_data():
    for file in glob.glob(r"D:\BTP\infant cry\donateacry_corpus_cleaned_and_updated_data\*"):
        label = os.path.basename(file)
        for audio in glob.glob(file+"\*.wav"):
            x.append(audio)
            if label!='hungry':
                label='nonHungry'
            y.append(label)


In [7]:
load_data()
print(len(x),len(y))

457 457


In [16]:
def shift_time(data, sampling_rate, shift_max, shift_direction):
    shift = np.random.randint(sampling_rate * shift_max)
    if shift_direction == 'right':
        shift = -shift
    elif shift_direction == 'both':
        direction = np.random.randint(0, 2)
        if direction == 1:
            shift = -shift
    augmented_data = np.roll(data, shift)
    # Set to silence for heading/ tailing
    if shift > 0:
        augmented_data[:shift] = 0
    else:
        augmented_data[shift:] = 0
    return augmented_data

In [17]:
def time_stretch(signal, time_stretch_rate):
    return librosa.effects.time_stretch(signal, time_stretch_rate)


In [18]:
def pitch_scale(signal, sr, num_semitones):
    return librosa.effects.pitch_shift(signal, sr, num_semitones)

In [19]:
import random
def random_gain(signal, min_factor=0.1, max_factor=0.12):
    gain_rate = random.uniform(min_factor, max_factor)
    augmented_signal = signal * gain_rate
    return augmented_signal

In [20]:
# Raw audio augmentation
augment_raw_audio = Compose(
    [
        AddGaussianNoise(min_amplitude=0.01, max_amplitude=0.2, p=1),
        PitchShift(min_semitones=-8, max_semitones=8, p=1),
        HighPassFilter(min_cutoff_freq=2000, max_cutoff_freq=4000, p=1)
    ]
)


In [43]:
aug_data=[]
aug_label=[]
i=0
for audio,label in zip(x,y):
    if label != 'hungry':
        signal,sr=librosa.load(audio)
        augmented_signal1=shift_time(signal,sr,2,'both')
        augmented_signal2=time_stretch(signal,0.5)
        augmented_signal3=pitch_scale(signal,sr,2)
#         augmented_signal4=shift_time(signal,sr,3,'right')
        augmented_signal5=random_gain(signal,2,4)
        augmented_signal6=augment_raw_audio(signal,sr)
        
        aug_data.append((augmented_signal1,sr))
        aug_label.append(label)
        aug_data.append((augmented_signal2,sr))
        aug_label.append(label)
        aug_data.append((augmented_signal3,sr))
#         aug_label.append(label)
#         aug_data.append((augmented_signal4,sr))
        aug_label.append(label)
        aug_data.append((augmented_signal5,sr))
        aug_label.append(label)
        aug_data.append((augmented_signal6,sr))
        aug_label.append(label)
#         sf.write("augmented_signal1.wav", augmented_signal, sr)


In [44]:
extracted_features=[]

def extractFeatures(X,Y):
    x=[]
    y=[]
    for audio,label in zip(X,Y):
        signal,sr=librosa.load(audio)
        feature = featureExtractor2(signal,sr)
        ar=np.resize(feature,(50,280))
        x.append(ar)
        y.append(label)
        extracted_features.append([feature,label])
    return x,y

In [45]:
X,Y=extractFeatures(x,y)

In [46]:
def extractFeatures2(X,Y):
    x=[]
    y=[]
    for audio,label in zip(X,Y):
        singal=audio[0]
        sr=audio[1]
        feature = featureExtractor2(signal,sr)
        ar=np.resize(feature,(50,280))
        x.append(ar)
        y.append(label)
        extracted_features.append([feature,label])
    return x,y

In [47]:
augX,augY=extractFeatures2(aug_data,aug_label)

In [48]:
for feature,label in zip(augX,augY):
    X.append(feature)
    Y.append(label)

In [89]:
print(len(X),len(Y))

832 832


In [90]:
#train test split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=0)
print(len(X_train),len(y_train),len(X_test),len(y_test))

665 665 167 167


In [91]:
length=len(X_train)
X_train=np.array(X_train).reshape(length,50,280)
y_train=np.array(y_train)
print(X_train.shape)
print(y_train.shape)

(665, 50, 280)
(665,)


In [92]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y_train=to_categorical(labelencoder.fit_transform(y_train))
y_train.shape

(665, 2)

In [93]:
length=len(X_test)
X_test=np.array(X_test).reshape(length,50,280)
y_test=np.array(y_test)
print(X_test.shape)
print(y_test.shape)

(167, 50, 280)
(167,)


In [94]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y_test=to_categorical(labelencoder.fit_transform(y_test))
y_test.shape

(167, 2)

In [96]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
print(X_train.shape,X_test.shape)

(665, 50, 280, 1) (167, 50, 280, 1)


In [97]:
# BUILDING MODEL

import tensorflow as tf
# print(tf.__version__)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow import keras as keras
from sklearn import metrics

# num_labels=y.shape[1]

def build_model(input_shape):
    model=Sequential()
    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu',input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((3,3), strides=(2,2),padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
    model.add(keras.layers.MaxPool2D((3,3), strides=(2,2),padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2,2), activation='relu'))
    model.add(keras.layers.MaxPool2D((2,2), strides=(2,2),padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten into 1D array
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64,activation='relu'))
    model.add(keras.layers.Dropout(0.5))

    # output layer
    model.add(keras.layers.Dense(2))
    model.add(Activation('sigmoid'))
    return model

input_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3])
model=build_model(input_shape)
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 48, 278, 32)       320       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 24, 139, 32)       0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 24, 139, 32)       128       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 22, 137, 32)       9248      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 11, 69, 32)        0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 11, 69, 32)        128       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 10, 68, 32)       

In [120]:
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# def f1_m(y_true, y_pred):
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [111]:
# Training
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

optimizer=keras.optimizers.Adam(learning_rate=0.001)
# model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

start=datetime.now()
# history = model.fit(x_train, y_train, validation_split=0.3, epochs=10, verbose=0)
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=32,epochs=15)

duration=datetime.now() - start
print("Training completed in time: ",duration)


Epoch 1/15
21/21 [==============================] - 12s 489ms/step - loss: 0.0188 - acc: 0.9945 - val_loss: 0.5111 - val_acc: 0.9102
Epoch 2/15
21/21 [==============================] - 9s 436ms/step - loss: 0.0283 - acc: 0.9895 - val_loss: 0.5310 - val_acc: 0.9102
Epoch 3/15
21/21 [==============================] - 10s 469ms/step - loss: 0.0264 - acc: 0.9901 - val_loss: 0.5579 - val_acc: 0.9042
Epoch 4/15
21/21 [==============================] - 9s 428ms/step - loss: 0.0244 - acc: 0.9889 - val_loss: 0.4479 - val_acc: 0.9102
Epoch 5/15
21/21 [==============================] - 9s 452ms/step - loss: 0.0113 - acc: 0.9990 - val_loss: 0.5369 - val_acc: 0.9102
Epoch 6/15
21/21 [==============================] - 9s 448ms/step - loss: 0.0175 - acc: 0.9932 - val_loss: 0.5090 - val_acc: 0.9162
Epoch 7/15
21/21 [==============================] - 10s 468ms/step - loss: 0.0188 - acc: 0.9950 - val_loss: 0.5665 - val_acc: 0.9162
Epoch 8/15
21/21 [==============================] - 10s 462ms/step - loss

In [115]:
loss, accuracy= model.evaluate(X_test, y_test, verbose=0)
# test_accuracy=model.evaluate(x_test,y_test,verbose=1)
# print(test_accuracy[1])
# test_accuracy
print("loss: ",loss)
print("f1 score: ",f1_score)
# print("precision: ",precision)
print("accuracy: ",accuracy)
# print("recall: ",recall)

loss:  0.6930208206176758
f1 score:  0.9273236393928528
accuracy:  0.916167676448822


In [117]:
y_pred = model.predict(X_test) 
y_pred = np.argmax(y_pred, axis = 1) 
label = np.argmax(y_test,axis = 1) 

from sklearn.metrics import confusion_matrix
print(confusion_matrix(label,y_pred))
# print("y_test:\n", label)
# print("y_pred:\n",y_pred) 


[[67  1]
 [13 86]]


In [118]:
model.save('models/model1.h5')

In [122]:
cnn1 = tf.keras.models.load_model("models/model1.h5")

In [149]:
filename='discomfort.wav'
prediction_feature=featureExtractor(filename)
prediction_feature=prediction_feature.reshape(-1,50,280,1)
print(prediction_feature.shape)

(1, 50, 280, 1)


In [150]:

# model.predict_classes(prediction_feature)
predicted_label=model.predict_classes(prediction_feature)
print(predicted_label)
predcited_class=labelencoder.inverse_transform(predicted_label)
predcited_class
# model.predict('/hungry')

[1]


array(['nonHungry'], dtype='<U9')